<a href="https://colab.research.google.com/github/Frederik-Roeckle/Rep_ImpactProcessComplexity/blob/master/Regression_PROVIDED_Authors_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import statsmodels.api as sm

In [16]:
drive.mount('/content/drive')

data = 'merged.csv'

# Freddys google drive path
data = '/content/drive/MyDrive/Colab Notebooks/Rep_ProcessMining/provided/merged.csv'

data = pd.read_csv(data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
datas = data.copy()

datas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1474 entries, 0 to 1473
Data columns (total 56 columns):
 #   Column                                                             Non-Null Count  Dtype  
---  ------                                                             --------------  -----  
 0   Date                                                               1474 non-null   object 
 1   Cases                                                              1474 non-null   int64  
 2   Distinct activities                                                1474 non-null   object 
 3   Number of distinct activities                                      1474 non-null   int64  
 4   Number of simple cycles in period                                  1474 non-null   int64  
 5   Average number of simple cycles per trace                          1474 non-null   float64
 6   Median number of simple cycles per trace                           1474 non-null   float64
 7   Number of activity repet

In [18]:
column_indices = [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
                  21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
                  41, 42, 43, 44, 45, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]

# Select the columns
datas_filtered = datas.iloc[:, column_indices]

In [19]:
datas_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1474 entries, 0 to 1473
Data columns (total 48 columns):
 #   Column                                                             Non-Null Count  Dtype  
---  ------                                                             --------------  -----  
 0   Cases                                                              1474 non-null   int64  
 1   Number of distinct activities                                      1474 non-null   int64  
 2   Number of simple cycles in period                                  1474 non-null   int64  
 3   Average number of simple cycles per trace                          1474 non-null   float64
 4   Median number of simple cycles per trace                           1474 non-null   float64
 5   Number of activity repetitions in period                           1474 non-null   int64  
 6   Average number of activity repetitions per trace                   1474 non-null   float64
 7   Median number of activit

In [20]:
datas['File'].value_counts()

File
BPI_Challenge_2019_metrics.csv                868
BPIC15_4_metrics.csv                           65
BPIC15_5_metrics.csv                           65
BPIC15_3_metrics.csv                           63
2020_PermitLog_metrics.csv                     60
BPIC15_1_metrics.csv                           59
BPIC15_2_metrics.csv                           58
BPI_Challenge_2018_metrics.csv                 45
2020_InternationalDeclarations_metrics.csv     44
Hospital_log_metrics.csv                       39
2020_RequestForPayment_metrics.csv             32
2020_DomesticDeclarations_metrics.csv          30
2020_PrepaidTravelCost_metrics.csv             26
BPI_Challenge_2017_metrics.csv                 14
financial_log_metrics.csv                       6
Name: count, dtype: int64

In [21]:
import pandas as pd
import numpy as np

# Check for missing values
null_columns = datas_filtered.columns[datas_filtered.isnull().any()].tolist()
print("Columns with null values:")
print(null_columns)

# Check for infinite values
#inf_columns = datas_filtered.columns[np.isinf(datas_filtered).any()].tolist()
#print("Columns with infinite values:")
#print(inf_columns)


Columns with null values:
['Number of participants in period', 'Average number of participants per trace', 'Median number of participants per trace', 'Handover of work in period', 'Average handover of work per trace', 'Median handover of work per trace', 'Average handover of work from participant', 'Median handover of work from participant', 'Affinity']


### There are null values in some columns

In [22]:
# Drop rows where 'Affinity' is null
datas_filtered = datas_filtered.dropna(subset=['Affinity', 'Number of participants in period', 'Handover of work in period',
                                               'Median handover of work from participant', 'Average handover of work from participant',
                                               'Median handover of work per trace', 'Average handover of work per trace', 'Median number of participants per trace',
                                               'Average number of participants per trace'])

# Now 'df' is your dataframe with rows dropped where 'Affinity' is null

**FULL** **MODELS**


In [23]:
# Define the dependent variable (median throughput time)
y = datas_filtered["Median throughput time"]

# Define the independent variables (all other metrics)
X = datas_filtered.drop("Median throughput time", axis=1)

# Add a constant to the independent variables matrix
X = sm.add_constant(X)
y

0         572551.5
1         638583.0
2         606969.0
3         694697.0
4         786880.0
           ...    
1424    48098190.0
1425    66773640.0
1426    66773640.0
1427    66773640.0
1428    66773640.0
Name: Median throughput time, Length: 1363, dtype: float64

In [24]:
import pandas as pd
import statsmodels.api as sm

def forward_selection(X, y):
    initial_list = []
    included = list(initial_list)
    while True:
        changed=False
        excluded = list(set(X.columns)-set(included))
        best_aic = float('inf')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            if model.aic < best_aic:
                best_aic = model.aic
                best_feature = new_column
        if best_aic < float('inf'):
            included.append(best_feature)
            changed=True
        if not changed:
            break
    print('Final model R-squared: {:.6}'.format(model.rsquared))
    return included, model

def backward_elimination(X, y):
    included=list(X.columns)
    while True:
        changed=False
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        worst_aic = model.aic
        worst_feature = None
        for column in included:
            temp_included = included[:]
            temp_included.remove(column)
            temp_model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[temp_included]))).fit()
            if temp_model.aic < worst_aic:
                worst_aic = temp_model.aic
                worst_feature = column
        if worst_feature is not None:
            included.remove(worst_feature)
            changed=True
        if not changed:
            break
    print('Final model R-squared: {:.6}'.format(model.rsquared))
    return included, model

def stepwise_selection(X, y):
    included, model = forward_selection(X, y)
    included, model = backward_elimination(X[included], y)
    final_model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
    print('Final model R-squared: {:.6}'.format(final_model.rsquared))
    return included, final_model

In [25]:
result_forward, model_fw = forward_selection(X, y)
result_backward, model_el = backward_elimination(X, y)
result_stepwise, model_sw = stepwise_selection(X, y)

print(len(result_forward))
print(len(result_backward))
print(len(result_stepwise))

Final model R-squared: 0.988188
Final model R-squared: 0.988166
Final model R-squared: 0.988188
Final model R-squared: 0.988185
Final model R-squared: 0.988185
47
44
46


**SIGNIFICANT MODELS**

In [26]:
import pandas as pd
import statsmodels.api as sm

def forward_selection(X, y):
    initial_list = []
    included = list(initial_list)
    while True:
        excluded = list(set(X.columns) - set(included))
        if len(excluded) == 0:  # Check if 'excluded' is empty
            break
        best_aic = float('inf')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            if model.aic < best_aic:
                best_aic = model.aic
                best_feature = new_column
        included.append(best_feature)
    # Filtering step
    final_model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
    final_cols = final_model.pvalues.loc[final_model.pvalues.index.isin(included)]
    included = final_cols[final_cols < 0.001].index.tolist()
    print('Final model R-squared: {:.6}'.format(final_model.rsquared))
    return included

def backward_elimination(X, y):
    included = list(X.columns)
    while True:
        changed = False
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        worst_aic = model.aic
        worst_feature = None
        for column in included:
            temp_included = included[:]
            temp_included.remove(column)
            temp_model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[temp_included]))).fit()
            if temp_model.aic < worst_aic:
                worst_aic = temp_model.aic
                worst_feature = column
        if worst_feature is not None:
            included.remove(worst_feature)
            changed = True
        if not changed:
            break
    # Filtering step
    final_model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
    final_cols = final_model.pvalues.loc[final_model.pvalues.index.isin(included)]
    included = final_cols[final_cols < 0.001].index.tolist()
    print('Final model R-squared: {:.6}'.format(final_model.rsquared))
    return included

def stepwise_selection(X, y):
    included = forward_selection(X, y)
    included = backward_elimination(X[included], y)
    # Filtering step
    final_model = sm.OLS(y, sm.add_constant(X[included])).fit()  # Removed unnecessary DataFrame conversion
    final_cols = final_model.pvalues.loc[final_model.pvalues.index.isin(included)]
    included = final_cols[final_cols < 0.001].index.tolist()
    print('Final model R-squared: {:.6}'.format(final_model.rsquared))
    return included


result_forward = forward_selection(X, y)
result_backward = backward_elimination(X, y)
result_stepwise = stepwise_selection(X, y)

print(len(result_forward))
print(len(result_backward))
print(len(result_stepwise))

Final model R-squared: 0.988188
Final model R-squared: 0.988166
Final model R-squared: 0.988188
Final model R-squared: 0.985513
Final model R-squared: 0.985422
23
28
51


**MINIMAL MODELS**


In [27]:
best_variables = {'size':['Magnitude',
                          'Variety',
                          'Support',
                          'Time granularity',
                          'Minimum trace length',
                          'Average trace length',
                          'Maximum trace length'],

                  'variation':['Level of detail',
                               'Pentland task complexity ',
                               'Lempel-Ziv',
                               'Distinct traces',
                               'Structure'],

                  'distance':['Affinity',
                              'Deviation from random'],

                  'entropy':['Simple Variant Entropy',
                             'Simple Sequence Entropy',
                             'Simple Normalized Variant Entropy',
                             'Simple Normalized Sequence Entropy',
                             'Enriched Sequence Entropy',
                             'Enriched Variant Entropy',
                             'Enriched Normalized Sequence Entropy',
                             'Enriched Normalized Variant Entropy'],

                  'generic':['Cases',
                             'Number of activity repetitions in period ',
                             'Average number of simple cycles per trace',
                             'Median number of simple cycles per trace',
                             'Number of distinct activities',
                             'Number of simple cycles in period',
                             'Average number of activity repetitions per trace',
                             'Median number of activity repetitions per trace',
                             'Number of participants in period',
                             'Average number of participants per trace',
                             'Median number of participants per trace',
                             'Handover of work in period',
                             'Average handover of work per trace',
                             'Median handover of work per trace',
                             'Average handover of work from participant',
                             'Median handover of work from participant']}

In [28]:
import pandas as pd
import statsmodels.api as sm


def select_best_variable_per_category(X, y, categories):
    best_variables = {}
    for category, variables in categories.items():
        best_pval = float('inf')
        best_r2 = float('-inf')
        best_variable = None
        for variable in variables:
            if variable in X.columns:
                model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[variable]))).fit()
                if model.pvalues[1] < best_pval or (model.pvalues[1] == best_pval and model.rsquared > best_r2):
                    best_pval = model.pvalues[1]
                    best_r2 = model.rsquared
                    best_variable = variable
        best_variables[category] = best_variable
    return best_variables

def forward_selection(X, y, best_variables):
    initial_list = list(best_variables.values())
    included = list(initial_list)
    while True:
        changed=False
        excluded = list(set(X.columns)-set(included))
        best_aic = float('inf')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            if model.aic < best_aic:
                best_aic = model.aic
                best_feature = new_column
        if best_aic < float('inf'):
            included.append(best_feature)
            changed=True
        if not changed:
            break
    print('Final model R-squared: {:.6}'.format(model.rsquared))
    return included

def backward_elimination(X, y, best_variables):
    initial_list = list(best_variables.values())
    included = list(initial_list)
    while True:
        changed=False
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        worst_aic = model.aic
        worst_feature = None
        for column in included:
            temp_included = included[:]
            temp_included.remove(column)
            temp_model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[temp_included]))).fit()
            if temp_model.aic < worst_aic:
                worst_aic = temp_model.aic
                worst_feature = column
        if worst_feature is not None:
            included.remove(worst_feature)
            changed=True
        if not changed:
            break
    print('Final model R-squared: {:.6}'.format(model.rsquared))
    return included

def stepwise_selection(X, y, best_variables):
    included = forward_selection(X, y, best_variables)
    included = backward_elimination(X[included], y, best_variables)
    final_model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
    print('Final model R-squared: {:.6}'.format(final_model.rsquared))
    return included

# Call the function to select the best variable per category
best_variables = select_best_variable_per_category(X, y, best_variables)

# Call the forward, backward, and stepwise selection functions
result_forward = forward_selection(X, y, best_variables)
result_backward = backward_elimination(X, y, best_variables)
result_stepwise = stepwise_selection(X, y, best_variables)

print('Best variable for each category:')
print(best_variables)
print('Resulting features (forward):')
print(result_forward)
print('Resulting features (backward):')
print(result_backward)
print('Resulting features (stepwise):')
print(result_stepwise)

Final model R-squared: 0.988188
Final model R-squared: 0.931999
Final model R-squared: 0.988188
Final model R-squared: 0.931999
Final model R-squared: 0.931999
Best variable for each category:
{'size': 'Average trace length', 'variation': 'Distinct traces', 'distance': 'Affinity', 'entropy': 'Simple Normalized Sequence Entropy', 'generic': 'Median number of participants per trace'}
Resulting features (forward):
['Average trace length', 'Distinct traces', 'Affinity', 'Simple Normalized Sequence Entropy', 'Median number of participants per trace', 'Deviation from random', 'Median number of simple cycles per trace', 'Average handover of work from participant', 'Minimum trace length', 'Enriched Normalized Sequence Entropy', 'Average number of simple cycles per trace', 'Simple Normalized Variant Entropy', 'Median number of activity repetitions per trace', 'Average number of participants per trace', 'Enriched Normalized Sequence Entropy (exponential forgetting, k1)', 'Simple Normalized Seque

**ADDING** **THE** **DUMMMY** **VARIABLE**

In [29]:
def assign_value(y):
    if y == ('BPIC15_3_metrics.csv'or 'BPIC15_4_metrics.csv' or 'BPIC15_5_metrics.csv' or 'BPIC15_2_metrics.csv' or 'BPIC15_1_metrics.csv' or 'BPI_Challenge_2018_metrics.csv'):  # replace this condition with your own
        return 'Public Administration'
    if y == ('2020_PrepaidTravelCost_metrics.csv' or '2020_DomesticDeclarations_metrics.csv' or '2020_RequestForPayment_metrics.csv' or '2020_InternationalDeclarations_metrics.csv' or '2020_PermitLog_metrics.csv'):
        return 'Education'
    if y == 'Hospital_log_metrics.csv':
      return 'Healthcare'
    if y == 'BPI_Challenge_2017_metrics.csv':
      return 'Finance'
    if y == 'BPI_Challenge_2019_metrics.csv':
      return 'Manufacturing'

dummy_filtered = datas.copy()

# Apply the function to the 'y' column and assign the result to a new column 'x'
dummy_filtered['INDUSTRY_DUMMY_VARIABLE'] = datas['File'].apply(assign_value)

In [30]:
dummy_filtered['INDUSTRY_DUMMY_VARIABLE'].value_counts()

INDUSTRY_DUMMY_VARIABLE
Manufacturing            868
Public Administration     63
Healthcare                39
Education                 26
Finance                   14
Name: count, dtype: int64

In [31]:
dummy_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1474 entries, 0 to 1473
Data columns (total 57 columns):
 #   Column                                                             Non-Null Count  Dtype  
---  ------                                                             --------------  -----  
 0   Date                                                               1474 non-null   object 
 1   Cases                                                              1474 non-null   int64  
 2   Distinct activities                                                1474 non-null   object 
 3   Number of distinct activities                                      1474 non-null   int64  
 4   Number of simple cycles in period                                  1474 non-null   int64  
 5   Average number of simple cycles per trace                          1474 non-null   float64
 6   Median number of simple cycles per trace                           1474 non-null   float64
 7   Number of activity repet

In [32]:
column_indices = [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
                  21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
                  41, 42, 43, 44, 45, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56]

# Select the columns
dummy_filtered = dummy_filtered.iloc[:, column_indices]

dummy_filtered = dummy_filtered.dropna(subset=['Affinity', 'Number of participants in period', 'Handover of work in period',
                                               'Median handover of work from participant', 'Average handover of work from participant',
                                               'Median handover of work per trace', 'Average handover of work per trace', 'Median number of participants per trace',
                                               'Average number of participants per trace'])


In [33]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()

dummy_filtered['INDUSTRY_DUMMY_VARIABLE'] = enc.fit_transform(dummy_filtered['INDUSTRY_DUMMY_VARIABLE'])

In [34]:
y = dummy_filtered["Median throughput time"]

# Define the independent variables (all other metrics)
X = dummy_filtered.drop("Median throughput time", axis=1)

# Add a constant to the independent variables matrix
X = sm.add_constant(X)

**FULL MODELS WITH DUMMY VARIABLE**

In [35]:
def forward_selection(X, y):
    initial_list = []
    included = list(initial_list)
    while True:
        changed=False
        excluded = list(set(X.columns)-set(included))
        best_aic = float('inf')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            if model.aic < best_aic:
                best_aic = model.aic
                best_feature = new_column
        if best_aic < float('inf'):
            included.append(best_feature)
            changed=True
        if not changed:
            break
    print('Final model R-squared: {:.6}'.format(model.rsquared))
    return included

def backward_elimination(X, y):
    included=list(X.columns)
    while True:
        changed=False
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        worst_aic = model.aic
        worst_feature = None
        for column in included:
            temp_included = included[:]
            temp_included.remove(column)
            temp_model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[temp_included]))).fit()
            if temp_model.aic < worst_aic:
                worst_aic = temp_model.aic
                worst_feature = column
        if worst_feature is not None:
            included.remove(worst_feature)
            changed=True
        if not changed:
            break
    print('Final model R-squared: {:.6}'.format(model.rsquared))
    return included

def stepwise_selection(X, y):
    included = forward_selection(X, y)
    included = backward_elimination(X[included], y)
    final_model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
    print('Final model R-squared: {:.6}'.format(final_model.rsquared))
    return included

In [36]:
result_forward = forward_selection(X, y)
result_backward = backward_elimination(X, y)
result_stepwise = stepwise_selection(X, y)

print(len(result_forward))
print('resulting features (forward):')
print(result_forward)
print(len(result_backward))
print('resulting features (backward):')
print(result_backward)
print(len(result_stepwise))
print('resulting features (stepwise):')
print(result_stepwise)

Final model R-squared: 0.9884
Final model R-squared: 0.988392
Final model R-squared: 0.9884
Final model R-squared: 0.988359
Final model R-squared: 0.988359
48
resulting features (forward):
['Affinity', 'Simple Normalized Variant Entropy', 'Deviation from random', 'Enriched Normalized Sequence Entropy', 'Average handover of work per trace', 'Average number of participants per trace', 'Median number of participants per trace', 'Level of detail', 'Lempel-Ziv', 'Number of simple cycles in period', 'Maximum trace length', 'Number of activity repetitions in period', 'Median number of simple cycles per trace', 'Median number of activity repetitions per trace', 'Simple Sequence Entropy', 'Structure', 'Pentland task complexity', 'INDUSTRY_DUMMY_VARIABLE', 'Number of distinct activities', 'Enriched Normalized Sequence Entropy (linear forgetting)', 'Simple Normalized Sequence Entropy (linear forgetting)', 'Time granularity', 'Simple Normalized Sequence Entropy (exponential forgetting, k1)', 'Dist

**SIGNIFICANT MODELS WITH DUMMY**

In [37]:
def forward_selection(X, y):
    initial_list = []
    included = list(initial_list)
    while True:
        changed = False
        excluded = list(set(X.columns) - set(included))
        best_aic = float('inf')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            if model.aic < best_aic:
                best_aic = model.aic
                best_feature = new_column
        if best_aic < float('inf'):
            included.append(best_feature)
            changed = True
        if not changed:
            break
    # Filtering step
    final_model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
    final_cols = final_model.pvalues.loc[final_model.pvalues.index.isin(included)]
    included = final_cols[final_cols < 0.001].index.tolist()
    print('Final model R-squared: {:.6}'.format(final_model.rsquared))
    return included

def backward_elimination(X, y):
    included = list(X.columns)
    while True:
        changed = False
        model = sm.OLS(y, sm.add_constant(X[included])).fit()  # Removed unnecessary DataFrame conversion
        worst_aic = model.aic
        worst_feature = None
        for column in included:
            temp_included = included[:]
            temp_included.remove(column)
            temp_model = sm.OLS(y, sm.add_constant(X[temp_included])).fit()  # Removed unnecessary DataFrame conversion
            if temp_model.aic < worst_aic:
                worst_aic = temp_model.aic
                worst_feature = column
        if worst_feature is not None:
            included.remove(worst_feature)
            changed = True
        if not changed:
            break
    # Filtering step
    final_model = sm.OLS(y, sm.add_constant(X[included])).fit()
    final_cols = final_model.pvalues.loc[final_model.pvalues.index.isin(included)]
    included = final_cols[final_cols < 0.001].index.tolist()
    print('Final model R-squared: {:.6}'.format(final_model.rsquared))
    return included

def stepwise_selection(X, y):
    included = forward_selection(X, y)
    included = backward_elimination(X[included], y)
    # Filtering step
    final_model = sm.OLS(y, sm.add_constant(X[included])).fit()  # Removed unnecessary DataFrame conversion
    final_cols = final_model.pvalues.loc[final_model.pvalues.index.isin(included)]
    included = final_cols[final_cols < 0.001].index.tolist()
    print('Final model R-squared: {:.6}'.format(final_model.rsquared))
    return included

result_forward = forward_selection(X, y)
result_backward = backward_elimination(X, y)
result_stepwise = stepwise_selection(X, y)

print(len(result_forward))
print(len(result_backward))
print(len(result_stepwise))

Final model R-squared: 0.9884
Final model R-squared: 0.988392
Final model R-squared: 0.9884
Final model R-squared: 0.976669
Final model R-squared: 0.974088
19
26
10


**MINIMAL MODELS WITH DUMMY**

In [38]:
best_variables = {'size':['Magnitude',
                          'Variety',
                          'Support',
                          'Time granularity',
                          'Minimum trace length',
                          'Average trace length',
                          'Maximum trace length'],

                  'variation':['Level of detail',
                               'Pentland task complexity ',
                               'Lempel-Ziv',
                               'Distinct traces',
                               'Structure'],

                  'distance':['Affinity',
                              'Deviation from random'],

                  'entropy':['Simple Variant Entropy',
                             'Simple Sequence Entropy',
                             'Simple Normalized Variant Entropy',
                             'Simple Normalized Sequence Entropy',
                             'Enriched Sequence Entropy',
                             'Enriched Variant Entropy',
                             'Enriched Normalized Sequence Entropy',
                             'Enriched Normalized Variant Entropy'],

                  'generic':['Cases',
                             'Number of activity repetitions in period ',
                             'Average number of simple cycles per trace',
                             'Median number of simple cycles per trace',
                             'Number of distinct activities',
                             'Number of simple cycles in period',
                             'Average number of activity repetitions per trace',
                             'Median number of activity repetitions per trace',
                             'Number of participants in period',
                             'Average number of participants per trace',
                             'Median number of participants per trace',
                             'Handover of work in period',
                             'Average handover of work per trace',
                             'Median handover of work per trace',
                             'Average handover of work from participant',
                             'Median handover of work from participant']}

In [39]:
import pandas as pd
import statsmodels.api as sm

def select_best_variable_per_category(X, y, categories):
    best_variables = {}
    for category, variables in categories.items():
        best_pval = float('inf')
        best_r2 = float('-inf')
        best_variable = None
        for variable in variables:
            if variable in X.columns:
                model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[variable]))).fit()
                if model.pvalues[1] < best_pval or (model.pvalues[1] == best_pval and model.rsquared > best_r2):
                    best_pval = model.pvalues[1]
                    best_r2 = model.rsquared
                    best_variable = variable
        best_variables[category] = best_variable
    return best_variables

def forward_selection(X, y, best_variables):
    initial_list = list(best_variables.values())
    included = list(initial_list)
    while True:
        changed=False
        excluded = list(set(X.columns)-set(included))
        best_aic = float('inf')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            if model.aic < best_aic:
                best_aic = model.aic
                best_feature = new_column
        if best_aic < float('inf'):
            included.append(best_feature)
            changed=True
        if not changed:
            break
    print('Final model R-squared: {:.6}'.format(model.rsquared))
    return included

def backward_elimination(X, y, best_variables):
    initial_list = list(best_variables.values())
    included = list(initial_list)
    while True:
        changed=False
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        worst_aic = model.aic
        worst_feature = None
        for column in included:
            temp_included = included[:]
            temp_included.remove(column)
            temp_model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[temp_included]))).fit()
            if temp_model.aic < worst_aic:
                worst_aic = temp_model.aic
                worst_feature = column
        if worst_feature is not None:
            included.remove(worst_feature)
            changed=True
        if not changed:
            break
    print('Final model R-squared: {:.6}'.format(model.rsquared))
    return included

def stepwise_selection(X, y, best_variables):
    included = forward_selection(X, y, best_variables)
    included = backward_elimination(X[included], y, best_variables)
    final_model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
    print('Final model R-squared: {:.6}'.format(final_model.rsquared))
    return included

# Call the function to select the best variable per category
best_variables = select_best_variable_per_category(X, y, best_variables)

# Call the forward, backward, and stepwise selection functions
result_forward = forward_selection(X, y, best_variables)
result_backward = backward_elimination(X, y, best_variables)
result_stepwise = stepwise_selection(X, y, best_variables)

print('Best variable for each category:')
print(best_variables)
print('Resulting features (forward):')
print(result_forward)
print('Resulting features (backward):')
print(result_backward)

Final model R-squared: 0.9884
Final model R-squared: 0.931999
Final model R-squared: 0.9884
Final model R-squared: 0.931999
Final model R-squared: 0.931999
Best variable for each category:
{'size': 'Average trace length', 'variation': 'Distinct traces', 'distance': 'Affinity', 'entropy': 'Simple Normalized Sequence Entropy', 'generic': 'Median number of participants per trace'}
Resulting features (forward):
['Average trace length', 'Distinct traces', 'Affinity', 'Simple Normalized Sequence Entropy', 'Median number of participants per trace', 'Deviation from random', 'Median number of simple cycles per trace', 'Average handover of work from participant', 'Minimum trace length', 'Enriched Normalized Sequence Entropy', 'Average number of simple cycles per trace', 'Simple Normalized Variant Entropy', 'Median number of activity repetitions per trace', 'Average number of participants per trace', 'INDUSTRY_DUMMY_VARIABLE', 'Enriched Normalized Sequence Entropy (exponential forgetting, k1)', '